In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
print(os.listdir('/kaggle/input/he_challenge_data/data'))
os.chdir('/kaggle/input/he_challenge_data/data')

# Any results you write to the current directory are saved as output.

['test.csv', 'test', 'train', 'train.csv', 'sample_submission.csv']


In [2]:
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')
train_df.head()

,image_id,category
0,0,77
1,1,81
2,2,52
3,3,72
4,4,58


In [3]:
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Dense
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [4]:
train_path='train/'
test_path='test/'

train_images=[]
train_labels=[]

for i in range(len(train_df.image_id)):
    img=cv2.imread(train_path+str(train_df.image_id[i])+'.jpg')
    img=cv2.resize(img,(224,224))
    train_images.append(img)
    train_labels.append(train_df.category[i])

In [5]:
train_path='train/'
test_path='test/'

test_images=[]
test_labels=[]

for i in range(len(test_df.image_id)):
    img=cv2.imread(test_path+str(test_df.image_id[i])+'.jpg')
    img=cv2.resize(img,(224,224))
    test_images.append(img)
    test_labels.append(test_df.category[i])

In [6]:
train_images=np.array(train_images)
test_images=np.array(test_images)

print('Train shape: {}'.format(train_images.shape))
print('Test shape: {}'.format(test_images.shape))

Train shape: (18540, 224, 224, 3)
Test shape: (2009, 224, 224, 3)


In [7]:
train_images=train_images.reshape(train_images.shape[0],224,224,3)
test_images=test_images.reshape(test_images.shape[0],224,224,3)

print('Train shape: {}'.format(train_images.shape))
print('Test shape: {}'.format(test_images.shape))

Train shape: (18540, 224, 224, 3)
Test shape: (2009, 224, 224, 3)


In [8]:
train_gen = ImageDataGenerator(featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True)

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(train_images,to_categorical(train_labels),random_state=42)

print('X-Train shape: {}'.format(X_test.shape))
print('X-Test shape: {}'.format(X_test.shape))
print('Y-Train shape: {}'.format(y_train.shape))
print('Y-Test shape: {}'.format(y_test.shape))

X-Train shape: (4635, 224, 224, 3)
X-Test shape: (4635, 224, 224, 3)
Y-Train shape: (13905, 103)
Y-Test shape: (4635, 103)


In [10]:
vgg=VGG16(weights='imagenet',include_top=True)
vgg.summary()

553467904/553467096 [==============================] - 9s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_po

In [11]:
model=Sequential()
for layer in vgg.layers[:-1]:
    model.add(layer)

for layer in model.layers[:]:
    layer.trainable=False
model.add(Dense(103,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       295168    
__________

In [12]:
history=model.fit(X_train,y_train,epochs=10,batch_size=32,validation_data=(X_test,y_test),callbacks=[EarlyStopping(min_delta=0.01)])

Train on 13905 samples, validate on 4635 samples
Epoch 1/10
13905/13905 [==============================] - 52s 4ms/step - loss: 1.2348 - acc: 0.7298 - val_loss: 0.6268 - val_acc: 0.8300
Epoch 2/10
13905/13905 [==============================] - 50s 4ms/step - loss: 0.2177 - acc: 0.9367 - val_loss: 0.3810 - val_acc: 0.8889
Epoch 3/10
13905/13905 [==============================] - 49s 4ms/step - loss: 0.0790 - acc: 0.9785 - val_loss: 0.3296 - val_acc: 0.9046
Epoch 4/10
13905/13905 [==============================] - 49s 4ms/step - loss: 0.0448 - acc: 0.9899 - val_loss: 0.2606 - val_acc: 0.9213
Epoch 5/10
13905/13905 [==============================] - 49s 4ms/step - loss: 0.0631 - acc: 0.9837 - val_loss: 0.2695 - val_acc: 0.9228


In [13]:
submission=pd.read_csv('sample_submission.csv')
submission.head()

,image_id,category
0,18540,0
1,18541,0
2,18542,0
3,18543,0
4,18544,0


In [14]:
test_id=[]
test_pred=[]

for i in submission.image_id:
    img=cv2.resize(cv2.imread('test/'+str(i)+'.jpg'),(224,224))
    img=np.expand_dims(img,axis=0)
    test_id.append(i)
    test_pred.append(int(model.predict_classes(img)))

In [15]:
final_submission=pd.DataFrame({'image_id':test_id,'category':test_pred})
final_submission.head()

,image_id,category
0,18540,18
1,18541,40
2,18542,97
3,18543,51
4,18544,51


In [16]:
final_submission.to_csv('/kaggle/final_submission.csv',index=False)